## Demo for SDK

This example shows how to use SDK to deploy a task. The demo notebook includes the following steps:
- [initialization](#initialization)
- [show available hardware info list](#show-available-hardware-information)
- [choose hardware config](#choose-hardware-config)
- [get `job_source_uri`](#get-job_source_uri)
- [estimate payment amount](#estimate-payment-amount)
- [create task to get `task_uuid`](#create-task)
- [submit payment](#submit-payment)
- [validate payment](#validate-payment-to-deploy-task)
- [follow up task status](#follow-up-task-status-optional)
- [show result](#show-result)



### Initialization

#### get an `API_KEY`

- For test version, get `API_KEY` in dashboard page: https://orchestrator-test.swanchain.io
- For prod version, get `API_KEY` in dashboard page: https://orchestrator.swanchain.io

If use this repository to test on your local machine, add `sys.path.insert(0, '..')` at the beginning, and run code in the root directory of this repository.

To use this SDK, you need to add environment file `.env` in your local directory, including the following parameters (`PK` is private key):

```
API_KEY=
WALLET=
PK=
```

In [1]:
import sys
sys.path.insert(0, '..') 

import os
import time
import dotenv
import json
dotenv.load_dotenv()
from swan import SwanAPI

dev_url = "https://swanhub-cali.swanchain.io"
# dev_url = "http://127.0.0.1:5008"

# Initialize the Swan Service
# dev
swan_api = SwanAPI(api_key=os.getenv("API_KEY"), environment=dev_url)
# prod
# swan_api = SwanAPI(api_key=os.getenv("API_KEY"))

from swan.contract.swan_contract import SwanContract

contract = SwanContract(os.getenv('PK'), swan_api.contract_info)

In [2]:
r = swan_api.contract_info
print(json.dumps(r, indent=2))

{
  "client_contract_address": "0x9c5397F804f6663326151c81bBD82bb1451059E8",
  "payment_contract_address": "0xB48c5D1c025655BA79Ac4E10C0F19523dB97c816",
  "rpc_url": "https://rpc-atom-internal.swanchain.io",
  "swan_token_contract_address": "0x91B25A65b295F0405552A4bbB77879ab5e38166c"
}


### Show available hardware information

In [3]:
hardwares = swan_api.get_hardware_config()
hardwares_info = [hardware.to_dict() for hardware in hardwares if hardware.status == "available"] 
hardwares_info

[{'id': 0,
  'name': 'C1ae.small',
  'description': 'CPU only · 2 vCPU · 2 GiB',
  'type': 'CPU',
  'reigion': ['Quebec-CA', 'North Carolina-US'],
  'price': '0.0',
  'status': 'available'},
 {'id': 1,
  'name': 'C1ae.medium',
  'description': 'CPU only · 4 vCPU · 4 GiB',
  'type': 'CPU',
  'reigion': ['Quebec-CA', 'North Carolina-US'],
  'price': '1.0',
  'status': 'available'},
 {'id': 12,
  'name': 'G1ae.small',
  'description': 'Nvidia 3080 · 4 vCPU · 8 GiB',
  'type': 'GPU',
  'reigion': ['Quebec-CA', 'North Carolina-US'],
  'price': '10.0',
  'status': 'available'},
 {'id': 13,
  'name': 'G1ae.medium',
  'description': 'Nvidia 3080 · 8 vCPU · 16 GiB',
  'type': 'GPU',
  'reigion': ['Quebec-CA', 'North Carolina-US'],
  'price': '11.0',
  'status': 'available'},
 {'id': 20,
  'name': 'Hpc1ae.small',
  'description': 'Nvidia 3090 · 4 vCPU · 8 GiB',
  'type': 'GPU',
  'reigion': ['Quebec-CA'],
  'price': '14.0',
  'status': 'available'},
 {'id': 21,
  'name': 'Hpc1ae.medium',
  'desc

### Choose hardware config

choose the hardware with its config name or hardware id

in hardware config, `cfg_name` and `hardware_id` will be used in the steps to deploy task (create task and submit payment)

In [4]:
cfg_name = 'C1ae.small' #"G1ae.medium"
hardware = [hardware for hardware in hardwares if hardware.name == cfg_name][0]
print(f"{hardware.name=}, {hardware.id=}, {hardware.region}")

hardware_id = hardware.id
print(f"The chosen {hardware_id=}")

hardware.name='C1ae.small', hardware.id=0, ['Quebec-CA', 'North Carolina-US']
The chosen hardware_id=0


In [5]:
hardware_id

0

### Get `job_source_uri`

We can use the `get_source_uri` interface to get a `job_source_uri` for creating task.

In [6]:

job_source_uri = swan_api.get_source_uri(
    repo_uri='https://github.com/alphaflows/tetris-docker-image.git',
    hardware_id=hardware_id,
    wallet_address=os.getenv('WALLET')
)

In [7]:
if job_source_uri and job_source_uri.get('data'):
    job_source_uri = job_source_uri['data']['job_source_uri']

In [8]:
job_source_uri

'https://plutotest.acl.swanipfs.com/ipfs/Qmd34gRU45ipwvSjbLpDk6YS4ZHcfAtfkGUm6dQiduWB1n'

### Estimate Payment amount

In [9]:
duration_hour = 1 # hour
amount = swan_api.estimate_payment(hardware_id, duration_hour)
print(amount)

0


### Create task

This step uses above information to create task, and get `task_uuid`, which is useful in submit payment step.

In [10]:
duration=3600*duration_hour

result = swan_api.create_task(
    cfg_name=cfg_name, 
    region='Quebec-CA', 
    start_in=300, 
    duration=duration, 
    job_source_uri=job_source_uri,
    paid=amount,
    wallet_address=os.getenv('WALLET'),
)
print(json.dumps(result, indent=2))
task_uuid = result['data']['task']['uuid']

{
  "data": {
    "task": {
      "comments": null,
      "created_at": 1719260493,
      "end_at": 1719264093,
      "id": 4350,
      "leading_job_id": null,
      "refund_amount": null,
      "refund_wallet": "0x00165Ca5Ab193c4D2709F94A5418165bA04394E6",
      "source": "v2",
      "start_at": 1719260493,
      "start_in": 300,
      "status": "initialized",
      "task_detail": {
        "amount": 0.0,
        "bidder_limit": 3,
        "created_at": 1719260493,
        "duration": 3600,
        "end_at": 1719264093,
        "hardware": "C1ae.small",
        "job_result_uri": null,
        "job_source_uri": "https://plutotest.acl.swanipfs.com/ipfs/Qmd34gRU45ipwvSjbLpDk6YS4ZHcfAtfkGUm6dQiduWB1n",
        "price_per_hour": "0.0",
        "requirements": {
          "hardware": "None",
          "hardware_type": "CPU",
          "memory": "2",
          "region": "Quebec-CA",
          "vcpu": "2"
        },
        "start_at": 1719260493,
        "status": "paid",
        "storage_so

### Submit Payment

This step is using `task_uuid`, `hardware_id` and `duration` to submit payment via **ClientPayment** contract.

In [11]:
# tx_hash = contract.submit_payment(task_uuid, hardware_id, duration)
# print(tx_hash)

### Validate Payment to deploy task

This step will validate the payment and then make task eligible for assigning if validation successful

In [12]:
# if result_validation := swan_api.validate_payment(
#     tx_hash=tx_hash,
#     task_uuid=task_uuid
# ):
#     print(json.dumps(result_validation, indent=2))
# else:
#     print('validation failed')

### Make Payment to deploy task

This step will make the payment and then make task eligible for assigning if payment successful

In [13]:
if result_validation := swan_api.make_payment(task_uuid, hardware_id, duration):
    print(json.dumps(result_validation, indent=2))
else:
    print('validation failed')

{'tx_hash': '0xd052869cb81af490cd0a5cdd49147710ad4c784a283aa551e5f49b4f2240e0b4', 'task_uuid': 'd4108d78-b0af-49fd-aa0f-cfadeeacc07d'}
{
  "data": {
    "task": {
      "comments": null,
      "created_at": 1719260493,
      "end_at": 1719264093,
      "id": 4350,
      "leading_job_id": null,
      "refund_amount": null,
      "refund_wallet": "0x00165Ca5Ab193c4D2709F94A5418165bA04394E6",
      "source": "v2",
      "start_at": 1719260493,
      "start_in": 300,
      "status": "created",
      "task_detail": {
        "amount": 0.0,
        "bidder_limit": 3,
        "created_at": 1719260493,
        "duration": 3600,
        "end_at": 1719264093,
        "hardware": "C1ae.small",
        "job_result_uri": null,
        "job_source_uri": "https://plutotest.acl.swanipfs.com/ipfs/Qmd34gRU45ipwvSjbLpDk6YS4ZHcfAtfkGUm6dQiduWB1n",
        "price_per_hour": "0.0",
        "requirements": {
          "hardware": "None",
          "hardware_type": "CPU",
          "memory": "2",
          "r

### follow up task status (optional)
The following step is optional, shows information when waiting for task being deployed.

In [14]:
# Check task info
info = swan_api.get_deployment_info(task_uuid=task_uuid)
print(json.dumps(info, indent=2))

{
  "data": {
    "computing_providers": [],
    "jobs": [],
    "task": {
      "comments": null,
      "created_at": 1719260493,
      "end_at": 1719264093,
      "id": 4350,
      "leading_job_id": null,
      "refund_amount": null,
      "refund_wallet": "0x00165Ca5Ab193c4D2709F94A5418165bA04394E6",
      "source": "v2",
      "start_at": 1719260493,
      "start_in": 300,
      "status": "created",
      "task_detail": {
        "amount": 0.0,
        "bidder_limit": 3,
        "created_at": 1719260493,
        "duration": 3600,
        "end_at": 1719264093,
        "hardware": "C1ae.small",
        "job_result_uri": null,
        "job_source_uri": "https://plutotest.acl.swanipfs.com/ipfs/Qmd34gRU45ipwvSjbLpDk6YS4ZHcfAtfkGUm6dQiduWB1n",
        "price_per_hour": "0.0",
        "requirements": {
          "hardware": "None",
          "hardware_type": "CPU",
          "memory": "2",
          "region": "Quebec-CA",
          "vcpu": "2"
        },
        "start_at": 1719260493,
  

### Show result

`job_real_uri` is for show the result of application you deployed.  
You can put it into the web browser to view application.

In [22]:
r = swan_api.get_real_url(task_uuid)
print(r)

['https://fzqs78rhba.dev2.crosschain.computer']


In [16]:
# r = swan_api.terminate_task(task_uuid)
# print(r)

In [24]:
claim_review = swan_api.claim_review(task_uuid)
print(claim_review)

{'data': {'error_code': 1012}, 'message': "task_uuid='d4108d78-b0af-49fd-aa0f-cfadeeacc07d' status (completed) not `finished`, cannot claim review", 'status': 'failed'}


In [25]:
import requests
import json

headers = {
    'Content-Type': 'application/json',
}

response = requests.get(r[0], headers=headers)

try:
    print(json.dumps(response.json(), indent=4))
except Exception as e:
    print(e)
    print(response)


SSLError: HTTPSConnectionPool(host='fzqs78rhba.dev2.crosschain.computer', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:997)')))